# Machine Learning: Smart o Ultra?

## Introducción

### La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra. En este proyecto, se ha evaluado el desempeño de diferentes modelos de clasificación para predecir la variable objetivo a partir de un conjunto de datos. El objetivo principal ha sido desarrollar un modelo con la mayor exactitud posible, superando el umbral de 0.75 en los conjuntos de validación y prueba. Para lograrlo, se probaron tres modelos: RandomForestClassifier, DecisionTreeClassifier y LogisticRegression, ajustando diversos hiperparámetros y evaluando la exactitud en los conjuntos de entrenamiento, validación y prueba. El análisis se centra en la comparación del rendimiento de cada modelo, destacando cuál cumple mejor con los requisitos del proyecto y cuál se adapta de forma óptima a los datos disponibles.

In [28]:
#Importamos las librerías

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [29]:
# Leemos el dataset

df = pd.read_csv('users_behavior.csv')

In [30]:
print(df.info())
print()
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
5   58.0   344.56      21.0  15823.37         0
6   57.0   431.64      20.0   3738.90         1
7   15.0   132.40       6.0  21911.60         0
8    7.0    43.39       3.0   2538.67         1
9   90.0   665.41      38.0  17358.61         0


In [31]:
# Segmentamos los datos en un conjunto de entrenamiento, uno de validación y uno de prueba

df_train, df_test = train_test_split(df, test_size=0.25, random_state=54321)
df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=54321)

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']


## Modelo DecisionTreeClassifier

In [35]:
# Investigamos la calidad de diferentes modelos cambiando los hiperparámetros
# Creamos un bucle para max_depth de 1 a 5

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=54321, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    print("max_depth =", depth, ": ", accuracy)

# Predecimos el conjunto de prueba con el mejor modelo
model = DecisionTreeClassifier(random_state=54321, max_depth=4)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)

# Calculamos la exactitud del modelo
accuracy_test = accuracy_score(target_test, predictions_test)
print("Exactitud en el conjunto de prueba:", accuracy_test)


max_depth = 1 :  0.7661691542288557
max_depth = 2 :  0.802653399668325
max_depth = 3 :  0.8076285240464345
max_depth = 4 :  0.8175787728026535
max_depth = 5 :  0.8109452736318408
Exactitud en el conjunto de prueba: 0.7810945273631841


##### En las pruebas de validación el modelo de arbol de decisión deja de mejorar a partir de la profundidad 4 alcanzado un 0.8175 de exactitud y en el conjunto de prueba la exactitud bajó a 0.7810

## Modelo RandomForestClassifier

In [36]:
# Investigamos la calidad de diferentes modelos cambiando los hiperparámetros

best_score = 0
best_est = 0

for est in range(10, 101):
    model = RandomForestClassifier(random_state=54321, n_estimators=est)
    model.fit(features_train, target_train)
    score = model.score(features_valid, target_valid)
    if score > best_score:
        best_score = score
        best_est = est

print('La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}'.format(best_est, best_score))

# Predecimos el conjunto de prueba con el mejor modelo
model = RandomForestClassifier(random_state=54321, n_estimators=12)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
print('Exactitud en el conjunto de prueba:', accuracy_score(target_test, predictions_test))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 12): 0.8192371475953566
Exactitud en el conjunto de prueba: 0.7761194029850746


##### En este modelo random forest con el conjunto de validación el valor de n_estimators = 12 resultó ser el mejor dando la mejor exactitud de 0.8192 y la exactitud en el conjunto de prueba fue de 0.7761 un poco por debajo de la primera prueba.

## Modelo LogisticRegression

In [38]:
# Investigamos la calidad de diferentes modelos cambiando los hiperparámetros

model = LogisticRegression(random_state=54321, solver='liblinear')
model.fit(features_train, target_train)
score_train = model.score(features_train, target_train)
score_valid = model.score(features_valid, target_valid)
score_test = model.score(features_test, target_test)

print('Accuracy del modelo de regresión logística en el conjunto de entrenamiento:', score_train)
print('Accuracy del modelo de regresión logística en el conjunto de validación:', score_valid)
print('Accuracy del modelo de regresión logística en el conjunto de prueba:', score_test)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7122302158273381
Accuracy del modelo de regresión logística en el conjunto de validación: 0.7313432835820896
Accuracy del modelo de regresión logística en el conjunto de prueba: 0.6965174129353234


##### En el modelo de regresión logística la exactitud no cumple con el umbral minimo de 0.75 en ninguna de las pruebas, siendo un modelo no viable para nuestros objetivos.

## Conclusión

### El modelo con la mejor exactitud que pude crear fue el RandomForestClassifier, alcanzando una exactitud de 0.8192. Aunque su rendimiento disminuyó ligeramente en el conjunto de prueba, esto es completamente normal, ya que los datos de prueba son completamente desconocidos para el modelo. Por otro lado, el DecisionTreeClassifier mostró una menor exactitud en el conjunto de validación, pero superó al RandomForest en el conjunto de pruebas, aunque de manera muy ajustada. Finalmente, el modelo de LogisticRegression no cumplió con el umbral de 0.75 en ninguna de las pruebas, por lo que se considera no apto para los objetivos del proyecto.